In [1]:
import pandas as pd

import nltk
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pickle

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
# checking if there is any null value or not in a sheet
df.shape
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [4]:
# Deleting a link column and also making a sample of 53000
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
# Text Cleaning/ Text Preprocessing
df['text'] = df['text'].str.lower().replace(r'\w\s', ' ').replace(r'\n', ' ', regex=True) 

In [6]:
df.head(5)

,artist,song,text
0,Kid Rock,God Bless Saturday,"in the justle and grind, workin' nine to nine ..."
1,Children,Arabella Miller,little arabella miller \r had a fuzzy caterpi...
2,Billy Joel,River Of Dreams,in the middle of the night \r i go walking in...
3,Whitesnake,Come An' Get It,are you woman enough \r to take a man like me...
4,Linda Ronstadt,I've Never Been In Love Before,i've never been in love before \r now all at ...


In [7]:
stemmer = PorterStemmer()

In [8]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)
    

In [9]:
# tokenization("You are beautiful, beautiful")

In [10]:
# df['text']= df['text'].apply(lambda x: tokenization(x))

# df['text'] = df['text'].apply(lambda x: tokenization(x))

In [11]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [12]:
matrix = tfid.fit_transform(df['text'])

In [13]:
similar = cosine_similarity(matrix)

In [14]:
# similar[0]

In [15]:
# Recommendor function
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key= lambda x:x[1])
    song = []
    for s_id in distance[:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [16]:
pickle.dump(similar, open("similarity.pkl", "wb"))

In [17]:
pickle.dump(df, open("df.pkl", "wb"))